In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import  SelectKBest
from keras.layers import Dense, Flatten, Conv1D,LSTM
from sklearn.feature_selection import f_classif
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score

from keras import backend as K
from functools import partial 
import warnings 
import keras
import matplotlib.pyplot as plt
import scikitplot.plotters as skplt


In [2]:
real_users = pd.read_csv("C:\\Users\\hp\\FYP\\users.csv")
fake_users = pd.read_csv("C:\\Users\\hp\\FYP\\fusers.csv")


In [3]:
real_users = real_users.drop(["id", "name", "screen_name", "created_at", "url","lang", "time_zone","location", "default_profile", "default_profile_image", "geo_enabled", "profile_image_url", "profile_banner_url", "profile_use_background_image", "profile_background_image_url_https", "profile_text_color", "profile_image_url_https", "profile_sidebar_border_color", "profile_background_tile", "profile_sidebar_fill_color", "profile_background_image_url", "profile_background_color", "profile_link_color", "description"], axis=1)
fake_users = fake_users.drop(["id", "name", "screen_name", "created_at", "url", "lang", "time_zone","location", "default_profile", "default_profile_image", "geo_enabled", "profile_image_url", "profile_banner_url", "profile_use_background_image", "profile_background_image_url_https", "profile_text_color", "profile_image_url_https", "profile_sidebar_border_color", "profile_background_tile", "profile_sidebar_fill_color", "profile_background_image_url", "profile_background_color", "profile_link_color","description"], axis=1)

In [4]:
x = pd.concat([real_users,fake_users])
X = pd.DataFrame(x)

In [5]:
x.shape
x.head()

,statuses_count,followers_count,friends_count,favourites_count,listed_count
0,20370,5470,2385,145,52
1,3131,506,381,9,40
2,4024,264,87,323,16
3,40586,640,622,1118,32
4,2016,62,64,13,0


In [6]:
y = len (fake_users)*['0'] + len(real_users)*['1']

In [7]:
x.dtypes

statuses_count      int64
followers_count     int64
friends_count       int64
favourites_count    int64
listed_count        int64
dtype: object

In [8]:
er = pd.Series(y)
print ("ER",er)
X['label'] = pd.DataFrame(er)
label = {'Real':0, 'Fake':1}



ER 0       0
1       0
2       0
3       0
4       0
       ..
2813    1
2814    1
2815    1
2816    1
2817    1
Length: 2818, dtype: object


In [9]:
import random
from nltk.corpus import names

In [10]:

def gender_features(word):
    return {'last_letter': word[-1]}

gender_features('amylia')


{'last_letter': 'a'}

In [11]:
nltk.download('names')

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [12]:
names = ([(name, 'male') for name in names.words('male.txt')] +
         [(name, 'female') for name in names.words('female.txt')])

random.shuffle(names)

In [13]:
featuresets = [(gender_features(n), g) for (n,g) in names]

train_set, test_set = featuresets[500:], featuresets[:500]

classifier = nltk.NaiveBayesClassifier.train(train_set)

print (classifier.classify(gender_features('Anat')))

male


In [14]:
print(nltk.classify.accuracy(classifier, train_set))

0.7638366469639979


In [15]:
classifier.show_most_informative_features(10)

Most Informative Features
             last_letter = 'k'              male : female =     44.0 : 1.0
             last_letter = 'a'            female : male   =     36.0 : 1.0
             last_letter = 'f'              male : female =     15.8 : 1.0
             last_letter = 'p'              male : female =     12.4 : 1.0
             last_letter = 'v'              male : female =     11.1 : 1.0
             last_letter = 'o'              male : female =      9.6 : 1.0
             last_letter = 'm'              male : female =      9.5 : 1.0
             last_letter = 'd'              male : female =      9.1 : 1.0
             last_letter = 'r'              male : female =      6.6 : 1.0
             last_letter = 'g'              male : female =      4.9 : 1.0


In [16]:
def strToBinary(s):
    bin_conv = []
    for c in s:
        ascii_val = ord(c)
        binary_val = bin(ascii_val)
        bin_conv.append(binary_val[2:])
    num=''.join(bin_conv)
    return num


def binaryToDecimal(n):
    num = int (n)
    dec_value = 0
    base = 1
    temp = num
        
    while(temp):
             last_digit = temp % 10
             temp = int (temp / 10)
             dec_value = last_digit* base
             base = base*2
            
             return dec_value

In [17]:
def extract_features(x):
    lang_list = list(enumerate(np.unique(x['lang'])))   
    lang_dict = { name : i for i, name in lang_list }             
    x.loc[:,'lang_code'] = x['lang'].map( lambda x: lang_dict[x]).astype(int)    
    x.loc[:,'sex_code']=predict_sex(x['name'])
    feature_columns_to_use = ['statuses_count','followers_count','friends_count','favourites_count','listed_count']
    x = x.loc[:,feature_columns_to_use]
    
   

In [18]:
 x_train,x_test,y_train,y_test = train_test_split(np.array(x),np.array(y),shuffle=True,test_size=0.3)
  

In [19]:
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam


In [20]:
num_epochs = 10
num_classes = 2
batch_size = 256
input_shape = (1972,9)



In [21]:
y_train_binary = keras.utils.to_categorical(y_train)
y_test_binary = keras.utils.to_categorical(y_test)

In [22]:
x_train =x_train.reshape(x_train.shape[0],x_train.shape[1],1)
x_test =x_test.reshape(x_test.shape[0],x_test.shape[1],1)


In [23]:
model_cnn = Sequential()
model_cnn.add(Conv1D(64,(3), input_shape=(x_train.shape[1],1),activation='relu'))
model_cnn.add(Flatten())
model_cnn.add(Dense(256,activation='relu'))
model_cnn.add(Dense(num_classes, activation='softmax'))

In [24]:
model_cnn.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 4, 64)             256       
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 66,562
Trainable params: 66,562
Non-trainable params: 0
_________________________________________________________________


In [25]:
x_train=np.asarray(x_train).astype(np.float64)
y_train=np.asarray(y_train).astype(np.float64)
x_test=np.asarray(x_test).astype(np.float64)
y_test=np.asarray(y_test).astype(np.float64)

In [26]:
history=model_cnn.fit(x_train,y_train_binary,batch_size=batch_size,epochs=num_epochs,verbose=1,validation_data=(x_test,y_test_binary))

Epoch 1/10
8/8 [==============================] - 37s 126ms/step - loss: 71.5377 - accuracy: 0.5316 - val_loss: 95.0744 - val_accuracy: 0.9090
Epoch 2/10
8/8 [==============================] - 0s 6ms/step - loss: 17.0563 - accuracy: 0.9106 - val_loss: 89.1346 - val_accuracy: 0.8345
Epoch 3/10
8/8 [==============================] - 0s 6ms/step - loss: 14.7378 - accuracy: 0.8711 - val_loss: 52.1248 - val_accuracy: 0.8298
Epoch 4/10
8/8 [==============================] - 0s 6ms/step - loss: 5.9542 - accuracy: 0.8717 - val_loss: 27.3445 - val_accuracy: 0.8853
Epoch 5/10
8/8 [==============================] - 0s 6ms/step - loss: 2.7722 - accuracy: 0.8922 - val_loss: 27.9019 - val_accuracy: 0.9173
Epoch 6/10
8/8 [==============================] - 0s 6ms/step - loss: 3.3240 - accuracy: 0.9145 - val_loss: 23.5921 - val_accuracy: 0.8842
Epoch 7/10
8/8 [==============================] - 0s 6ms/step - loss: 1.6617 - accuracy: 0.9012 - val_loss: 18.7962 - val_accuracy: 0.9007
Epoch 8/10
8/8 [=====

In [27]:
y_pred_cnn=model_cnn.predict(x_test)
print(y_pred_cnn)

[[4.1582431e-03 9.9584168e-01]
 [6.6686323e-07 9.9999928e-01]
 [2.8630254e-01 7.1369749e-01]
 ...
 [9.9999630e-01 3.7507061e-06]
 [2.1141655e-13 1.0000000e+00]
 [1.7113784e-06 9.9999833e-01]]


In [ ]:
thm = y_pred_cnn[0,0]
tym= y_pred_cnn[0,1]

In [ ]:
a = input ("Enter the Name : ")
username=classifier.classify(gender_features(a))
print(username)
s=username
ln=[]
for i in s:
     k=0
     c=strToBinary(i)
     h=binaryToDecimal(c)
     ln.append(h)
name=sum(ln)
if username =="male":
    gender=0
else:
    gender=1
status_count=int(input("status_count:"))
followers_count=int(input("followers_count:"))
friends_count=int(input("friends_count:"))
favourites_count=int(input("favourites_count:"))


test = {"name":name,"gender":gender,"status_count":status_count,"followers_count":followers_count,"friends_count":friends_count,"favourites_count":favourites_count}
de=pd.DataFrame(test,index=[0])
de.to_csv("test.csv")
re=pd.read_csv("test.csv")
rs=re.iloc[:,1:9].values
print(rs)
print(rs.shape)

rs=rs.reshape(1,6,1)

In [ ]:
y_pred_cnn_input=model_cnn.predict(rs)
print(y_pred_cnn_input)

In [ ]:
first = y_pred_cnn_input.item(0)*100
second = y_pred_cnn_input.item(1)*100

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
lab = ['Genuine Account', 'Fake Account']
fea = [first, second]
ax.pie(fea,labels = lab,autopct = '%1.2f%%')
plt.show()